Our data was gathered by direct export from a data set available through [pushhift.io's collection of reddit comments](https://files.pushshift.io/reddit/comments/).  The data unfortunately came "raw", with no strong attempt to avoid conflit with any potential .csv separators, and so much attention had to be paid to cleaning throughout the process. Two month's of data were used, representing all reddit comments from August and September of 2017. August of 2017 contained 84,658,503 posts; September contained 83,165,192.  The use of two months was due to an external constraint; namely to make the data processing be even remotely feasible.  The months chosen were semi-arbitrary, they fulfilled two criteria of not being obviously "special" with respect to political activity (somewhat removed in time from the previous election, not obviously around any major political stories relative to what's possible, etc) and being far enough back in time that it was possible to have some semblence of removednesses from current ongoings in order to try to assess media spotlighting of specific communities. As mentioned earlier, the files are quite large, both between 7-8 GB in their compressed form.  As such, they are not provided in this repo; rather if one wishes to reproduce this work these files should be obtained from the link given above.  


Note that "post" here is being used in the conventional sense, interchangeable with "comment".  It is not uncommon within reddit to treat "posts" as the initial comment or item beginning a thread and all subsequent replies along that thread as "comments".  The comments here represent both of these.  

Each month's data was formatted, in the original compressed .bz2 file, such that a single line would correspond to a single reddit post containing the following features:

|Entry|Explanation|
|--:|:--|
|body|The post contents, exactly as made (more on this below).|
|score hidden|Deprecated (null).|
|archived|Deprecated (null).|
|author|The username of the comment's creator.|
|author_flair_text|A short string associated with a specific user within a specific subreddit.|
|downs|Deprecated (null).|
|created_utc|Time the post was authored|
|subreddit_id|Code identifying subreddit (subforum/community) the post was made to.|
|link_id|Contradictory explanations, perhaps changing over time.  Seemingly represents a link to the specific post. Unused.|
|parent_id|Identifies original post in a given thread.|
|score|The rating given to the post by other users.|
|retreived_on|The time the post was archived (presumably, unclear).|
|controversiality|Possibly deprecated, possibly still functioning as a flag to indicate a post received both many positive and negative ratings.  Unused.|
|gilded|Indicates that a post resulted in another user gifting the author with "reddit gold", access to premium reddit features for a given period of time.|
|id|Identifies the post.|
|subreddit|The community/subforum the post was made to.|
|ups|Deprecated (null).|
|distinguished|Deprecated (null).|
|author_flair_css_class|Represents a decorator a post author might have associated with their account on a given subreddit.|

Of these featueres, the fields initially chosen to preserve during decompression were 'body', 'id', 'author', 'created_utc','subreddit','retrieved_on',and 'parent_id'.  Ultimately only 'subreddit' and 'body' were used for analysis. 

Mindful of memory constraints, the basic plan for the files was then:
1. Open the .bz2 file to read line-by-line
2. Initialize "files written" counter and a "lines written" counter. 
3. Open a .csv file to write to, appending to its name the current value for the "files written" counter.  
4. Read a line from the .bz2 file.  If it passes the filtering criteria, explained below, write it to the open .csv file and increment the "lines written" counter by one.  
5. If the number of lines written has surpassed a pre-set size, chosen (perhaps too cautiously) as 1,000,000, close the .csv file, increment the "files written" counter by 1, and start writing to a new .csv file with that counter's value as the new ending of its name. 
6. Continue until the .bz2 file is empty.

This functionality is provided in the `dataprocessing` subpackage of the `ex_id_tools` package included in this repo, should one wish to use it directly, under the module name `unpack_zipped`.  It should work with at least the files written *up to* these ones, though it's not impossible that something will go wrong for files written later. 

There are two things to note here: 
1. To save time and memory, some lines were automatically filtered out, based on initial exploration of samples of the data.  This consisted of "users" that were in actuality bots providing some site-wide helpful function, as well as posts that had been deleted prior to being harvested into the original dataset.  
2. The comments were transcribed directly into the original data set. As such, every csv separator that *could* come up *did* come up; keep in mind for instance there are conceivably comments made in reddit of people giving examples of csv files with different separators.  A choice was made that seemed to minimize errors and the fieldnames were specified ahead of time to try to mitigate this, but the data comes in extremely messy regardless and problematic entries do exist, which must later be handled.  

We'll shortly see this data being put into a sql database for further manipulation; if resources allow, note this same data is available on [Google's BigQuery service](https://console.cloud.google.com/bigquery?project=first-mark-284422&redirect_from_classic=true&p=fh-bigquery&d=reddit_comments&page=dataset), though an interested party should note that size of the tables means that even modest queries will require significant data usage.




These csvs were then explored a bit to get a feel for what would be feasible.  It was determined that as part of this work time and resource constraints would restrict analysis to simply subreddit and body information; while the other details provided in these files is useful and will be fruitful for further analysis, we will now move to extracting only the desired information.  